In [31]:
!pip install plotly

In [32]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path
import os
import plotly.io as pio

# Thiết lập chế độ hiển thị cho Plotly trong Notebook
pio.templates.default = "plotly_white"
print("Đang sử dụng Plotly để tạo biểu đồ...")

# --- THIẾT LẬP ĐƯỜNG DẪN VÀ BIẾN ---
PROJECT_ROOT = Path(os.getcwd()).parent
PROCESSED_DIR = PROJECT_ROOT / 'processed'
# THAY ĐỔI: Sử dụng FIGURES_DIR thay cho REPORTS_DIR
FIGURES_DIR = PROJECT_ROOT / 'figures'

FIGURES_DIR.mkdir(exist_ok=True)
CITY_NAME = 'Paris'

print(f"Thư mục Report sẽ lưu tại: {FIGURES_DIR}")

Đang sử dụng Plotly để tạo biểu đồ...
Thư mục Report sẽ lưu tại: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4\figures


In [33]:
# --- 1. TẢI DỮ LIỆU CẦN THIẾT ---
print("\n--- TẢI DỮ LIỆU KPI ---")
try:
    # 3 files KPI với tiền tố kpi_
    df_supply = pd.read_csv(PROCESSED_DIR / 'kpi_supply_paris.csv')
    df_price = pd.read_csv(PROCESSED_DIR / 'kpi_price_paris.csv')
    df_room = pd.read_csv(PROCESSED_DIR / 'kpi_room_type_paris.csv') 
    
    # 2 files không có tiền tố kpi_
    df_price_nhood = pd.read_csv(PROCESSED_DIR / 'price_nhood_paris.csv') # Đã sửa tên file
    df_listings_all = pd.read_csv(PROCESSED_DIR / 'listings_all_valid_paris.csv', low_memory=False) 

    # Chuyển đổi cột ngày sang định dạng datetime
    df_supply['snapshot_date'] = pd.to_datetime(df_supply['snapshot_date'], format='%d %B, %Y')
    df_price['snapshot_date'] = pd.to_datetime(df_price['snapshot_date'], format='%d %B, %Y')
    df_room['snapshot_date'] = pd.to_datetime(df_room['snapshot_date'], format='%d %B, %Y')
    df_price_nhood['snapshot_date'] = pd.to_datetime(df_price_nhood['snapshot_date'], format='%d %B, %Y')
    df_listings_all['snapshot_date'] = pd.to_datetime(df_listings_all['snapshot_date'], format='%d %B, %Y')

except FileNotFoundError as e:
    print(f"❌ Lỗi: Không tìm thấy file KPI. Vui lòng kiểm tra lại file Paris_Summary_KPI.ipynb. Lỗi: {e}")
    # Nếu lỗi, không cần chạy tiếp
    raise
    
print("✅ Đã tải dữ liệu thành công.")


--- TẢI DỮ LIỆU KPI ---
✅ Đã tải dữ liệu thành công.


In [34]:
# =========================================================================
## 📈 1. KPI VỀ NGUỒN CUNG (paris_01_supply.pdf) - BIỂU ĐỒ CỘT
# =========================================================================
print("\nĐang tạo biểu đồ 1: Nguồn cung (Biểu đồ Cột)...")
df_supply_sorted = df_supply.sort_values('snapshot_date')

fig_supply = px.bar(
    df_supply_sorted, 
    x='snapshot_date', 
    y='total_listings',
    text='total_listings', 
    color_discrete_sequence=['#4B8B9B'], 
    title=f'1. Xu Hướng Tổng Nguồn Cung tại {CITY_NAME}'
)
fig_supply.update_layout(
    xaxis_title='', 
    yaxis_title='TỔNG LISTINGS',
    xaxis={'categoryorder': 'array', 'categoryarray': df_supply_sorted['snapshot_date'].unique()},
    bargap=0.1, 
)
fig_supply.update_traces(textposition='outside') 

fig_supply.show()
fig_supply.write_image(FIGURES_DIR / 'paris_01_supply.pdf') 
print("✅ Đã lưu paris_01_supply.pdf")


Đang tạo biểu đồ 1: Nguồn cung (Biểu đồ Cột)...


✅ Đã lưu paris_01_supply.pdf


In [35]:
# =========================================================================
## 📈 2. KPI VỀ GIÁ (paris_02_price.pdf) - BIỂU ĐỒ KHU VỰC
# =========================================================================
print("\nĐang tạo biểu đồ 2: Giá trung vị (Biểu đồ Khu vực)...")
df_price_sorted = df_price.sort_values('snapshot_date')

# Tạo biểu đồ Khu vực (Area Plot)
fig_price = px.area(
    df_price_sorted, 
    x='snapshot_date', 
    y='median_price', 
    markers=True, # Thêm markers tại các điểm dữ liệu
    title=f'H2. XU HƯỚNG GIÁ TRUNG VỊ (MEDIAN PRICE) TẠI {CITY_NAME.upper()}',
    labels={'median_price': 'GIÁ (USD)'},
    color_discrete_sequence=['#A2D794'], # Màu xanh lá nhạt cho Area Plot
)

# Thêm nhãn giá trị (text annotations) lên trên các điểm
for index, row in df_price_sorted.iterrows():
    fig_price.add_annotation(
        x=row['snapshot_date'],
        y=row['median_price'],
        # Định dạng giá trị $xx, làm tròn về số nguyên
        text=f'${int(row["median_price"])}',
        showarrow=False,
        yref='y',
        yshift=15, # Dịch chuyển lên trên marker
        font=dict(size=14, color='darkgreen', weight='bold') 
    )

# Điều chỉnh layout theo yêu cầu (định dạng trục x, tiêu đề)
fig_price.update_layout(
    xaxis_title='',
    yaxis_title='GIÁ (USD)',
    xaxis={'categoryorder': 'array', 'categoryarray': df_price_sorted['snapshot_date'].unique()},
    showlegend=False,
)

fig_price.show()
# Lưu file vào thư mục figures/
fig_price.write_image(FIGURES_DIR / 'paris_02_price.pdf') 
print("✅ Đã lưu paris_02_price.pdf")


Đang tạo biểu đồ 2: Giá trung vị (Biểu đồ Khu vực)...


✅ Đã lưu paris_02_price.pdf


In [36]:
# =========================================================================
## 📊 3. KPI VỀ CẤU TRÚC PHÒNG (paris_03_room_type.pdf) - BIỂU ĐỒ CỘT NHÓM
# =========================================================================
print("\nĐang tạo biểu đồ 3: Cấu trúc phòng (Biểu đồ Cột Nhóm)...")
df_room_sorted = df_room.sort_values('snapshot_date')

# Định nghĩa màu sắc theo hình ảnh tham khảo
color_map_room = {
    'Entire home/apt': '#54A895', # Xanh ngọc
    'Private room': '#8796C8',    # Xanh dương nhạt
    'Shared room': '#E498B5',     # Hồng nhạt
    'Hotel room': '#F5A962'       # Cam nhạt
}

fig_room = px.bar(
    df_room_sorted, 
    x='snapshot_date', 
    y='percentage', 
    color='room_type',
    text=df_room_sorted['percentage'].apply(lambda x: f'{x:.1f}%'), # Định dạng hiển thị 1 chữ số thập phân
    color_discrete_map=color_map_room,
    title='H3. PHÂN BỔ LOẠI PHÒNG (%) THEO THỜI GIAN',
    labels={'percentage': 'TỶ LỆ (%)', 'room_type': 'LOẠI PHÒNG'},
    height=500,
    barmode='group' # Đảm bảo các cột được nhóm lại
)

# Đảm bảo các nhãn văn bản được đặt bên trên cột
fig_room.update_traces(textposition='outside')

fig_room.update_layout(
    xaxis_title='',
    yaxis_title='TỶ LỆ (%)',
    xaxis={'categoryorder': 'array', 'categoryarray': df_room_sorted['snapshot_date'].unique()},
    legend_title_text='LOẠI PHÒNG' # Đổi tên tiêu đề Legend
)

fig_room.show()
# Lưu file vào thư mục figures/
fig_room.write_image(FIGURES_DIR / 'paris_03_room_type.pdf') 
print("✅ Đã lưu paris_03_room_type.pdf")


Đang tạo biểu đồ 3: Cấu trúc phòng (Biểu đồ Cột Nhóm)...


✅ Đã lưu paris_03_room_type.pdf


In [37]:
# =========================================================================
## 📈 4. TOP 10 GIÁ (paris_04_top10_price.pdf) - BIỂU ĐỒ CỘT NGANG
# =========================================================================
print("\nĐang tạo biểu đồ 4: Top 10 khu vực có giá cao nhất...")
latest_date = df_price_nhood['snapshot_date'].max()
df_latest_nhood = df_price_nhood[df_price_nhood['snapshot_date'] == latest_date]

# Lọc Top 10 và sắp xếp giảm dần (đắt nhất lên đầu)
top_10_price = df_latest_nhood.sort_values(by='median_price', ascending=False).head(10)

# Tạo biểu đồ cột ngang
fig_top10 = px.bar(
    top_10_price, 
    x='median_price', # Trục x là giá
    y='neighbourhood_cleansed', # Trục y là khu vực
    orientation='h', # Cột ngang
    color='median_price', # Tô màu theo giá trị
    color_continuous_scale=px.colors.sequential.Reds, # Dải màu Đỏ
    title=f'H4. TOP 10 KHU VỰC ĐẮT ĐỎ NHẤT ({latest_date.strftime("%d %B, %Y")})',
    labels={'median_price': 'GIÁ TRUNG VỊ (USD)', 'neighbourhood_cleansed': 'KHU VỰC (NEIGHBOURHOOD)'},
)

# Sắp xếp trục Y (Khu vực) theo giá trị (giảm dần từ trên xuống)
fig_top10.update_layout(
    yaxis={'categoryorder': 'total ascending'}, # Đảo ngược thứ tự trục Y để giá trị cao nhất ở trên cùng
    xaxis_title='GIÁ TRUNG VỊ (USD)',
    yaxis_title='KHU VỰC (NEIGHBOURHOOD)'
)

fig_top10.show()
# Lưu file vào thư mục figures/
fig_top10.write_image(FIGURES_DIR / 'paris_04_top10_price.pdf') 
print("✅ Đã lưu paris_04_top10_price.pdf")


Đang tạo biểu đồ 4: Top 10 khu vực có giá cao nhất...


✅ Đã lưu paris_04_top10_price.pdf


In [38]:
# =========================================================================
## 📈 5. HEATMAP BIẾN ĐỘNG GIÁ (paris_05_price_heatmap.pdf)
# =========================================================================
print("\nĐang tạo biểu đồ 5: Heatmap Biến động Giá...")

# Chuẩn bị dữ liệu cho Heatmap: Tạo Pivot Table
df_heatmap_data = df_price_nhood.pivot_table(
    index='neighbourhood_cleansed', 
    columns='snapshot_date', 
    values='median_price'
)

# Đảm bảo các cột (ngày) được sắp xếp đúng thứ tự thời gian
df_heatmap_data = df_heatmap_data[df_price_nhood['snapshot_date'].unique()]

# Sắp xếp các khu vực theo giá trị trung bình của họ (tùy chọn)
df_heatmap_data['mean_price'] = df_heatmap_data.mean(axis=1)
df_heatmap_data = df_heatmap_data.sort_values(by='mean_price', ascending=False).drop(columns=['mean_price'])

# Chuyển đổi định dạng ngày tháng sang chuỗi ngắn gọn cho trục X
date_format = lambda d: d.strftime('%d %B, %Y')
df_heatmap_data.columns = [date_format(c) for c in df_heatmap_data.columns]


# Tạo Heatmap bằng Plotly Express
fig_heatmap = px.imshow(
    df_heatmap_data,
    text_auto=True, 
    aspect="auto",  
    color_continuous_scale=px.colors.sequential.Blues, 
    y=df_heatmap_data.index.tolist(),
    x=df_heatmap_data.columns.tolist(),
    labels=dict(x="THỜI ĐIỂM SNAPSHOT", y="KHU VỰC", color="GIÁ TRUNG VỊ (USD)"),
    height=800 
)

# --- PHẦN ĐÃ SỬA ĐỂ TẠO KHOẢNG SỐ CÁCH ĐỀU LỚN TRÊN TRỤC MÀU (COLOR BAR) ---
min_price = df_heatmap_data.values.min()
max_price = df_heatmap_data.values.max()
# Tạo các giá trị cách đều nhau 10 (hoặc 20) trong khoảng min/max
# np.arange sẽ tạo mảng số, ví dụ từ 70 đến 120 với bước nhảy 10.
tick_values = np.arange(
    start=np.floor(min_price / 10) * 10,  # Làm tròn xuống 10 gần nhất
    stop=np.ceil(max_price / 10) * 10 + 1, # Làm tròn lên 10 gần nhất
    step=10 # Khoảng cách giữa các số là 10
)

fig_heatmap.update_layout(
    title=f'H5. HEATMAP BIẾN ĐỘNG GIÁ THEO KHU VỰC (USD)',
    title_x=0.5, 
    xaxis_title='THỜI ĐIỂM SNAPSHOT',
    yaxis_title='KHU VỰC',
    coloraxis_colorbar=dict(
        title="GIÁ TRUNG VỊ (USD)",
        # Chỉ hiển thị các giá trị cách đều nhau (ví dụ: 70, 80, 90,...)
        tickvals=tick_values.tolist()
    )
)

fig_heatmap.show()
# Lưu file vào thư mục figures/
fig_heatmap.write_image(FIGURES_DIR / 'paris_05_price_heatmap.pdf') 
print("✅ Đã lưu paris_05_price_heatmap.pdf")


Đang tạo biểu đồ 5: Heatmap Biến động Giá...


✅ Đã lưu paris_05_price_heatmap.pdf


In [39]:
# =========================================================================
## 🗺️ 6. BẢN ĐỒ DỮ LIỆU GỐC (paris_06_map_labeled.pdf)
# =========================================================================
print("\nĐang tạo biểu đồ 6: Bản đồ Phân bố Listings Gốc...")

# Lấy dữ liệu từ snapshot mới nhất
df_latest_map = df_listings_all[df_listings_all['snapshot_date'] == df_listings_all['snapshot_date'].max()]

# Chỉ lấy một phần mẫu nhỏ (ví dụ: 1000 listings) từ snapshot mới nhất để tránh quá tải hiển thị
df_sample = df_latest_map.sample(n=1000, random_state=42) 

fig_map_labeled = px.scatter_mapbox(
    df_sample, 
    lat="latitude", 
    lon="longitude", 
    color="price", # Màu sắc dựa trên giá
    hover_name="neighbourhood_cleansed",
    hover_data=['room_type', 'price'], # Thông tin hiển thị khi di chuột
    color_continuous_scale=px.colors.sequential.Viridis, 
    # ĐÃ SỬA: Tăng zoom lên 11.5 để phóng to bản đồ
    zoom=11, 
    height=600,
    title=f'H6. BẢN ĐỒ PHÂN BỐ LISTINGS (MẪU) TẠI {CITY_NAME.upper()}'
)

# Cập nhật kiểu bản đồ
fig_map_labeled.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":50,"l":0,"b":0}, # Tối ưu hóa margin
    coloraxis_colorbar=dict(title='GIÁ ($)')
)

fig_map_labeled.show()
# Lưu file vào thư mục figures/
fig_map_labeled.write_image(FIGURES_DIR / 'paris_06_map_labeled.pdf') 
print("✅ Đã lưu paris_06_map_labeled.pdf")

print("\n--- CHÚC MỪNG! BẠN ĐÃ HOÀN TẤT VIỆC TẠO 6 BIỂU ĐỒ CHO PARIS ---")


Đang tạo biểu đồ 6: Bản đồ Phân bố Listings Gốc...


C:\Users\Admin\AppData\Local\Temp\ipykernel_14520\3785071988.py:12: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



✅ Đã lưu paris_06_map_labeled.pdf

--- CHÚC MỪNG! BẠN ĐÃ HOÀN TẤT VIỆC TẠO 6 BIỂU ĐỒ CHO PARIS ---
